In [1]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.impute import KNNImputer
from sklearn.linear_model import LinearRegression


# Load the AIS training data
ais_train = pd.read_csv('data/ais_train.csv', delimiter='|')

def haversine(lat1, lon1, lat2, lon2):
    # Haversine formula to calculate distance between two latitude-longitude points
    R = 6371  # Radius of Earth in kilometers
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = np.sin(dlat / 2) ** 2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

def calculate_bearing(lat1, lon1, lat2, lon2):
    # Calculate the bearing angle between two lat-long points
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    x = np.sin(dlon) * np.cos(lat2)
    y = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(dlon)
    return np.degrees(np.arctan2(x, y))

# Function to preprocess the data
def preprocess_ais_data(df):
    # Convert time column to datetime
    df['time'] = pd.to_datetime(df['time'])
    
    # Sort by vesselId and time
    df = df.sort_values(by=['vesselId', 'time'])
    
    df = df.dropna()
    # Handle NaN values with Linear Regression Imputation
    #df = linear_regression_impute(df)
    
    # Feature extraction: Time differences, velocity, and direction (heading)
    df['time_diff'] = df.groupby('vesselId')['time'].diff().dt.total_seconds() / 3600.0  # in hours
    df['lat_diff'] = df.groupby('vesselId')['latitude'].diff().fillna(0)  # Fill NaNs with 0
    df['lon_diff'] = df.groupby('vesselId')['longitude'].diff().fillna(0)  # Fill NaNs with 0
    df['distance'] = df.apply(lambda row: geodesic((row['latitude'], row['longitude']),
                                                   (row['latitude'] - row['lat_diff'],
                                                   row['longitude'] - row['lon_diff'])).km, axis=1)
    
    df['haversine_distance'] = df.apply(lambda row: haversine(row['latitude'], row['longitude'],
                                                              row['latitude'] - row['lat_diff'],
                                                              row['longitude'] - row['lon_diff']), axis=1)
    
    # Calculate speed (in km/h)
    df['speed'] = df['distance'] / df['time_diff']
    
    # Calculate acceleration (change in speed)
    df['acceleration'] = df.groupby('vesselId')['speed'].diff() / df['time_diff']
    
    # Calculate change in heading over time
    df['heading_diff'] = df.groupby('vesselId')['heading'].diff() / df['time_diff']
    
    # Calculate relative bearing (difference between course over ground and heading)
    df['relative_bearing'] = df['cog'] - df['heading']
    
    # Calculate turn rate (change in course over ground over time)
    df['turn_rate'] = df.groupby('vesselId')['cog'].diff() / df['time_diff']
    
    # Extract hour of the day
    df['hour_of_day'] = df['time'].dt.hour
    
    # Calculate cumulative distance traveled
    df['cumulative_distance'] = df.groupby('vesselId')['distance'].cumsum()
    
    # Calculate days since last port arrival (example)
    df['days_since_last_port'] = (df['time'] - df.groupby('vesselId')['time'].transform('min')).dt.total_seconds() / (3600 * 24)    
    
    # Additional feature engineering
    df['bearing_angle'] = df.apply(lambda row: calculate_bearing(row['latitude'] - row['lat_diff'], 
                                                                 row['longitude'] - row['lon_diff'], 
                                                                 row['latitude'], 
                                                                 row['longitude']), axis=1)
    df['cumulative_speed'] = df.groupby('vesselId')['speed'].cumsum()
    df['bearing_change_rate'] = df['bearing_angle'].diff() / df['time_diff']
    df['cumulative_heading_change'] = df.groupby('vesselId')['heading_diff'].cumsum()
    
    df['previous_latitude'] = df['latitude'] - df['lat_diff']
    df['previous_longitude'] = df['longitude'] - df['lon_diff']
    df['distance_from_prev_point'] = df['distance']
    df['time_gap_from_prev_point'] = df['time_diff']

    df['delta_cog'] = df.groupby('vesselId')['cog'].diff()
    df['delta_rot'] = df.groupby('vesselId')['rot'].diff()

    df['time_gap_from_start'] = (df['time'] - df.groupby('vesselId')['time'].transform('min')).dt.total_seconds() / 3600.0
    df['cumulative_time_difference'] = df.groupby('vesselId')['time_diff'].cumsum()

    # Fill NaNs resulting from differencing and rolling calculations
    df.fillna(0, inplace=True)

    
    return df

# Function to impute missing values using Linear Regression
def linear_regression_impute(df):
    # Isolate rows with missing latitude or longitude
    missing_lat = df['latitude'].isnull()
    missing_lon = df['longitude'].isnull()

    # Prepare the feature set for prediction
    feature_columns = ['time_diff', 'lat_diff', 'lon_diff', 'speed']
    
    # Train Linear Regression for latitude
    if missing_lat.any():
        # Train data: Drop rows where 'latitude' is missing and also drop NaNs in the feature columns
        lat_train = df[~missing_lat].dropna(subset=feature_columns)
        X_lat_train = lat_train[feature_columns]
        y_lat_train = lat_train['latitude']

        # Fit the model
        lat_model = LinearRegression()
        lat_model.fit(X_lat_train, y_lat_train)

        # Predict missing latitudes
        X_lat_missing = df[missing_lat][feature_columns].fillna(0)
        df.loc[missing_lat, 'latitude'] = lat_model.predict(X_lat_missing)

    # Train Linear Regression for longitude
    if missing_lon.any():
        # Train data: Drop rows where 'longitude' is missing and also drop NaNs in the feature columns
        lon_train = df[~missing_lon].dropna(subset=feature_columns)
        X_lon_train = lon_train[feature_columns]
        y_lon_train = lon_train['longitude']

        # Fit the model
        lon_model = LinearRegression()
        lon_model.fit(X_lon_train, y_lon_train)

        # Predict missing longitudes
        X_lon_missing = df[missing_lon][feature_columns].fillna(0)
        df.loc[missing_lon, 'longitude'] = lon_model.predict(X_lon_missing)

    return df

# Preprocess the training data
ais_train = preprocess_ais_data(ais_train)

# Select relevant features for training
#features = ['cog', 'sog', 'rot', 'heading', 'speed']
# Final feature list
features = ['cog', 'sog', 'rot', 'heading', 'speed', 'acceleration', 'heading_diff', 
            'relative_bearing', 'turn_rate', 'hour_of_day', 'haversine_distance', 
            'cumulative_distance', 'days_since_last_port', 'bearing_angle', 'cumulative_speed',
            'bearing_change_rate', 'cumulative_heading_change', 'previous_latitude', 
            'previous_longitude', 'distance_from_prev_point', 'time_gap_from_prev_point', 'delta_cog', 'delta_rot', 
            'time_gap_from_start', 'cumulative_time_difference']

target_lat = 'latitude'
target_lon = 'longitude'

# Use MinMaxScaler to scale the features
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(ais_train[features])

# Prepare the train data
X_train = scaled_features
y_train_lat = ais_train[target_lat].values
y_train_lon = ais_train[target_lon].values


In [2]:
#from sklearn.ensemble import RandomForestRegressor

# Train a Random Forest model for latitude
#rf_lat = RandomForestRegressor(n_estimators=50, random_state=42)
#rf_lat.fit(X_train, y_train_lat)

# Train a Random Forest model for longitude
#rf_lon = RandomForestRegressor(n_estimators=50, random_state=42)
#rf_lon.fit(X_train, y_train_lon)

# Print model summary
#print("Random Forest models trained for latitude and longitude.")


In [3]:
import xgboost as xgb

# Train an XGBoost model for latitude
xgb_lat = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
xgb_lat.fit(X_train, y_train_lat)

# Train an XGBoost model for longitude
xgb_lon = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
xgb_lon.fit(X_train, y_train_lon)

# Print model summary
print("XGBoost models trained for latitude and longitude.")


XGBoost models trained for latitude and longitude.


In [4]:
# Load the AIS test data
ais_test = pd.read_csv('data/ais_test.csv', delimiter=',')

# Convert time to datetime in test data
ais_test['time'] = pd.to_datetime(ais_test['time'])

# Initialize empty lists to store the predicted lat/lon for the test set
#rf_predicted_latitudes = [np.nan] * len(ais_test)
#rf_predicted_longitudes = [np.nan] * len(ais_test)

xgb_predicted_latitudes = [np.nan] * len(ais_test)
xgb_predicted_longitudes = [np.nan] * len(ais_test)

# Iterate over each vessel in the test set
for vessel_id in ais_test['vesselId'].unique():
    # Extract the historical data for this vessel from the training data
    vessel_train_data = ais_train[ais_train['vesselId'] == vessel_id]
    
    if len(vessel_train_data) > 0:
        # Preprocess this vessel's historical data (scale)
        vessel_features = scaler.transform(vessel_train_data[features])
        
        # Predict using Random Forest
        #rf_lat_pred = rf_lat.predict(vessel_features)
        #rf_lon_pred = rf_lon.predict(vessel_features)
        
        # Predict using XGBoost
        xgb_lat_pred = xgb_lat.predict(vessel_features)
        xgb_lon_pred = xgb_lon.predict(vessel_features)
        
        # Find the indices in ais_test for this vessel
        indices = ais_test.index[ais_test['vesselId'] == vessel_id].tolist()
        
        # Append the last prediction as the next position
        for idx in indices:
            #rf_predicted_latitudes[idx] = rf_lat_pred[-1]
            #rf_predicted_longitudes[idx] = rf_lon_pred[-1]
            
            xgb_predicted_latitudes[idx] = xgb_lat_pred[-1]
            xgb_predicted_longitudes[idx] = xgb_lon_pred[-1]

    else:
        # If no historical data, append NaN or fallback value
        #rf_predicted_latitudes.append(np.nan)
        #rf_predicted_longitudes.append(np.nan)
        
        xgb_predicted_latitudes.append(np.nan)
        xgb_predicted_longitudes.append(np.nan)

# Store the predictions back into ais_test dataframe
#ais_test['latitude_predicted'] = rf_predicted_latitudes
#ais_test['longitude_predicted'] = rf_predicted_longitudes

ais_test['latitude_predicted'] = xgb_predicted_latitudes
ais_test['longitude_predicted'] = xgb_predicted_longitudes

# Save predictions to CSV for submission
ais_test[['ID', 'longitude_predicted', 'latitude_predicted']].to_csv('predictions_xgb.csv', index=False)


In [5]:
# Option 1: Select the XGBoost model predictions
#ais_test['final_latitude'] = ais_test['xgb_predicted_latitude']
#ais_test['final_longitude'] = ais_test['xgb_predicted_longitude']

# Option 2: Ensemble by averaging Random Forest and XGBoost predictions
#ais_test['latitude_predicted'] = (ais_test['rf_predicted_latitude'] + ais_test['xgb_predicted_latitude']) / 2
#ais_test['longitude_predicted'] = (ais_test['rf_predicted_longitude'] + ais_test['xgb_predicted_longitude']) / 2

# Save final predictions to CSV
#ais_test[['ID', 'longitude_predicted', 'latitude_predicted']].to_csv('final_predictions.csv', index=False)


In [6]:
from geopy.distance import geodesic

# Function to calculate weighted geodetic distance
def calculate_weighted_distance(predictions, ground_truth):
    weights = [0.3, 0.25, 0.2, 0.15, 0.1]
    total_distance = 0
    total_weight = 0
    
    for i, weight in enumerate(weights):
        pred_lat = predictions[f'final_latitude_day_{i+1}']
        pred_lon = predictions[f'final_longitude_day_{i+1}']
        true_lat = ground_truth[f'latitude_day_{i+1}']
        true_lon = ground_truth[f'longitude_day_{i+1}']
        
        # Calculate geodetic distance for each day
        distance = geodesic((true_lat, true_lon), (pred_lat, pred_lon)).km
        total_distance += weight * distance
        total_weight += weight
    
    return total_distance / total_weight  # Weighted average distance

# Apply this function to calculate the final evaluation metric

